# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\pos_add_20240823_file2.csv")
df.shape

(20, 9)

In [3]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [4]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,TERRITORY,TSM,REGION,Brand
0,FC Home Center Candelaria,Standalone,For Opening,23-Aug-24,MDLY3Z9,Bicol,Penfrancia Ayo,South Luzon,FC Home
1,Jr Mx Pacific Centre Binondo,Standalone,For Opening,19-Aug-24,MXI3S6W,NNCR,Sander Maghirang,North Luzon,MemoXpress
2,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,19-Aug-24,M4XIFGS,Bicol,Penfrancia Ayo,South Luzon,MemoXpress
3,Jr Mx Waltermart Dasmarinas,Waltermart Dasmarinas,POS Operational,22-Aug-24,M4WS9TG,Cavite Dasma,Kathleen Incoy,South Luzon,MemoXpress
4,Jr Mx Super Metro Lapu Lapu,Super Metro Lapu Lapu,POS Operational,13-Aug-24,MT10J4X,Cebu North,Rona Jay Sinangote,VISAYAS,MemoXpress


In [5]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'TERRITORY', 'TSM', 'REGION', 'BrandStores'],
      dtype='object')

In [6]:
df['BrandStores'].value_counts()

BrandStores
JCE Fone      15
MemoXpress     4
FC Home        1
Name: count, dtype: int64

Check whether we have duplicate purple key in the data provided by the SIL team

In [7]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MDLY3Z9    1
MXI3S6W    1
M4XIFGS    1
M4WS9TG    1
MT10J4X    1
MK4WN52    1
MZ3HN9Y    1
M1KXDBO    1
MFH3EK5    1
MG0ODBX    1
MTOSI0A    1
M7YJ12E    1
MEYLI7K    1
MXUHR78    1
MV25D0H    1
MIHX81Z    1
M23CNPV    1
MTUEP6I    1
MLQ6VA0    1
M38O0L2    1
Name: count, dtype: int64

In [8]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,FC Home,FC Home Center Candelaria,Standalone,For Opening,23-Aug-24,MDLY3Z9,South Luzon,Penfrancia Ayo
1,MemoXpress,Jr Mx Pacific Centre Binondo,Standalone,For Opening,19-Aug-24,MXI3S6W,North Luzon,Sander Maghirang
2,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,19-Aug-24,M4XIFGS,South Luzon,Penfrancia Ayo
3,MemoXpress,Jr Mx Waltermart Dasmarinas,Waltermart Dasmarinas,POS Operational,22-Aug-24,M4WS9TG,South Luzon,Kathleen Incoy
4,MemoXpress,Jr Mx Super Metro Lapu Lapu,Super Metro Lapu Lapu,POS Operational,13-Aug-24,MT10J4X,VISAYAS,Rona Jay Sinangote
5,JCE Fone,JCE Fonehub Waltermart Tanauan,Waltermart Tanauan,For Opening,24-Aug-24,MK4WN52,South Luzon,Jan Jan Dela Cruz
6,JCE Fone,Infinix Concept Store Waltermart Tanauan,Waltermart Tanauan,For Opening,24-Aug-24,MZ3HN9Y,South Luzon,Jan Jan Dela Cruz
7,JCE Fone,Infinix Waltermart Bicutan,Waltermart Bicutan,For Opening,24-Aug-24,M1KXDBO,South Luzon,Grace Belarmino
8,JCE Fone,Tecno Waltermart Bicutan,Waltermart Bicutan,For Opening,24-Aug-24,MFH3EK5,South Luzon,Grace Belarmino
9,JCE Fone,Infinix Waltermart Dasma,Waltermart Dasmarinas,POS Operational,22-Aug-24,MG0ODBX,South Luzon,Kathleen Incoy


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      20 non-null     object
 1   POS NAME         20 non-null     object
 2   STORE LOCATION   20 non-null     object
 3   STATUS           20 non-null     object
 4   DATE ACTIVATION  20 non-null     object
 5   PURPLE KEY       20 non-null     object
 6   REGION           20 non-null     object
 7   TSM              20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


In [10]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [11]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Candelaria,Standalone,For Opening,23-Aug-24,MDLY3Z9,South Luzon,Penfrancia Ayo
1,MemoXpress,Jr Mx Pacific Centre Binondo,Standalone,For Opening,19-Aug-24,MXI3S6W,North Luzon,Sander Maghirang
2,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,19-Aug-24,M4XIFGS,South Luzon,Penfrancia Ayo
3,MemoXpress,Jr Mx Waltermart Dasmarinas,Waltermart Dasmarinas,POS Operational,22-Aug-24,M4WS9TG,South Luzon,Kathleen Incoy
4,MemoXpress,Jr Mx Super Metro Lapu Lapu,Super Metro Lapu Lapu,POS Operational,13-Aug-24,MT10J4X,VISAYAS,Rona Jay Sinangote


In [12]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

ValueError: time data "23-Aug-24" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


ValueError: time data "23-Aug-24" doesn't match format "%B %d, %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [14]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

object
23-Aug-24


AttributeError: Can only use .dt accessor with datetimelike values

In [15]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

In [16]:
print(df['DateOpened'].dtype)
# datetime64[ns]

datetime64[ns]


In [17]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-08-23 00:00:00


In [18]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    20 non-null     object
 1   POS            20 non-null     object
 2   StoreLocation  20 non-null     object
 3   Status         20 non-null     object
 4   DateOpened     20 non-null     object
 5   PurpleKey      20 non-null     object
 6   Region         20 non-null     object
 7   TSM            20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


In [19]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    20 non-null     object        
 1   POS            20 non-null     object        
 2   StoreLocation  20 non-null     object        
 3   Status         20 non-null     object        
 4   DateOpened     20 non-null     datetime64[ns]
 5   PurpleKey      20 non-null     object        
 6   Region         20 non-null     object        
 7   TSM            20 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 1.4+ KB


In [21]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Candelaria,Standalone,For Opening,2024-08-23,MDLY3Z9,South Luzon,Penfrancia Ayo
1,MemoXpress,Jr Mx Pacific Centre Binondo,Standalone,For Opening,2024-08-19,MXI3S6W,North Luzon,Sander Maghirang
2,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,2024-08-19,M4XIFGS,South Luzon,Penfrancia Ayo
3,MemoXpress,Jr Mx Waltermart Dasmarinas,Waltermart Dasmarinas,POS Operational,2024-08-22,M4WS9TG,South Luzon,Kathleen Incoy
4,MemoXpress,Jr Mx Super Metro Lapu Lapu,Super Metro Lapu Lapu,POS Operational,2024-08-13,MT10J4X,VISAYAS,Rona Jay Sinangote


In [22]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [23]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=00b8b14f-8f35-4ad4-b4c2-fe4770fae9e8>

In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID d1859ccc-a002-453c-a493-3b37e5a19f04 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(892, 8)


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID f6840cba-cdfc-4f08-b6e4-357dde3a24c4 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [26]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Faith Glorious Fit              8
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince Retail                  11
Prince and Aviel                3
RC

In [27]:
dfsil.shape

(892, 8)

In [28]:
dfsil.columns

Index(['BrandStores', 'POS', 'StoreLocation', 'Status', 'DateOpened',
       'PurpleKey', 'Region', 'TSM'],
      dtype='object')

In [29]:
dfsil[dfsil['PurpleKey'] =='MA9VNCJ']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
599,Rulls,Rulls Prince Hypermarket Camotes,Standalone,POS Operational,2024-08-18,MA9VNCJ,VISAYAS,Rona Jay Sinangote


In [30]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 912 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 973d36b3-2ea1-43df-bba6-0cf6d691fb44 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (912, 8)


In [32]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
902,JCE Fone,Infinix LCC Iriga,LCC Mall Iriga,For Opening,2024-08-24,MXUHR78,South Luzon,Penfrancia Ayo
903,JCE Fone,Infinix Festival Mall,Festival Mall,For Opening,2024-08-24,MV25D0H,South Luzon,Grace Belarmino
904,JCE Fone,Power Switch Festival Mall,Festival Mall,For Opening,2024-08-24,MIHX81Z,South Luzon,Grace Belarmino
905,JCE Fone,Infinix LCC Goa,LCC Mall Goa,For Opening,2024-08-24,M23CNPV,South Luzon,Penfrancia Ayo
906,JCE Fone,Inifinix Yashano Mall Naga,Yashano Mall Naga,For Opening,2024-08-24,MTUEP6I,South Luzon,Penfrancia Ayo
907,JCE Fone,Infinix Central Mall Sta Cruz,Central Mall Sta Cruz,For Opening,2024-08-24,MLQ6VA0,South Luzon,Aubrey Lalong Isip
908,JCE Fone,Realme Robinsons Town Los Banos,Robinsons Town Los Banos,For Opening,2024-08-24,M38O0L2,South Luzon,Aubrey Lalong Isip
909,MemoXpress,Jr Mx Super Metro Lapu Lapu,Super Metro Lapu Lapu,POS Operational,2024-08-13,MT10J4X,VISAYAS,Rona Jay Sinangote
910,MemoXpress,Jr Mx Waltermart Dasmarinas,Waltermart Dasmarinas,POS Operational,2024-08-22,M4WS9TG,South Luzon,Kathleen Incoy
911,JCE Fone,Infinix Waltermart Dasma,Waltermart Dasmarinas,POS Operational,2024-08-22,MG0ODBX,South Luzon,Kathleen Incoy


In [33]:
df['BrandStores'].value_counts().sort_index()

BrandStores
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       110
Faith Glorious                  9
Faith Glorious Fit              8
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JCE Fone                       15
JSL                             7
JTG                             1
Jhunpyo                        12
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     58
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince Retail                  11
Pr

In [34]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,6,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,7,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
30,854,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
29,852,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
31,861,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
32,862,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
21,774,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
20,773,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
25,790,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
24,789,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [35]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [37]:
df[df['PurpleKey']== 'M4XIFGS']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
893,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,2024-08-19,M4XIFGS,South Luzon,Penfrancia Ayo


In [38]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3c6a8a02-003c-45e7-90e4-9d347f135ef1>

In [39]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=52b9acab-108a-4e5d-937b-0776f35b45da>

In [40]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=703e5488-424d-4bed-926a-310b7d51e85f>

In [41]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID b1283bde-69dc-455b-a3b9-bfe869bc9c2b successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(912, 8)

In [42]:
df[df['BrandStores']=='VMax Mobile'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
534,VMax Mobile,Lets Mobile Phone and accessories Store,Divimall Cabuyao,POS Operational,2024-07-27,MSNZ3XL,South Luzon,Aubrey Lalong Isip
535,VMax Mobile,Vmax Central Binan,Central Mall Binan,POS Operational,2024-07-27,MZEO85E,South Luzon,Aubrey Lalong Isip
536,VMax Mobile,Vmax Waltermart Sta Rosa,Waltermart Starosa,POS Operational,2024-07-27,MJ0WZAK,South Luzon,Aubrey Lalong Isip
537,VMax Mobile,Vmax Central Balibago,Victory Central Mall Balibago,POS Operational,2024-07-27,MWN8FVI,South Luzon,Aubrey Lalong Isip
538,VMax Mobile,Delle General Merch Harmony Mall,Harmony Village Mall,POS Operational,2024-07-27,M3YOW2R,South Luzon,Aubrey Lalong Isip
539,VMax Mobile,Vmax Central Binan 2,Central Mall Binan,POS Operational,2024-07-27,ME4NBT7,South Luzon,Aubrey Lalong Isip
540,VMax Mobile,Delle General Merch Target Mall,Target Mall Balibago,For Opening,2024-08-02,MLX9FJP,South Luzon,Aubrey Lalong Isip
541,VMax Mobile,Realme Target Mall Balibago,Target Mall Balibago,For Opening,2024-08-02,MXACV0S,South Luzon,Aubrey Lalong Isip
542,VMax Mobile,Vmax Waltermart Cabuyao,Waltermart Cabuyao,For Opening,2024-08-02,MKPNC4B,South Luzon,Aubrey Lalong Isip
543,VMax Mobile,St Vincent Telecom Target Mall,Target Mall Starosa,For Opening,2024-08-02,MVTZGDH,South Luzon,Aubrey Lalong Isip


In [43]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Pasig,Standalone,POS Operational,2024-08-02,MOJ34ME,ENCR,Martin Lin
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center San ...,Standalone,POS Operational,2024-08-02,MB3YSES,CAMANAVA,Paul Palonpon
3,Home Along,Home Along Talavera,Standalone,POS Operational,2024-08-15,MZKJOGD,North Luzon,Rhea Geronimo
4,JCE Fone,Infinix Concept Store Waltermart Tanauan,Waltermart Tanauan,For Opening,2024-08-24,MZ3HN9Y,South Luzon,Jan Jan Dela Cruz
5,JCE Fone,JCE Fonehub Waltermart Tanauan,Waltermart Tanauan,For Opening,2024-08-24,MK4WN52,South Luzon,Jan Jan Dela Cruz
6,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
7,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
8,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
9,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None


In [45]:
df.sort_values('DateOpened', ascending=False).head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
142,JCE Fone,Xioami LCC Iriga,LCC Mall Iriga,For Opening,2024-08-24,MEYLI7K,South Luzon,Penfrancia Ayo
135,JCE Fone,Infinix Central Mall Dasma,Central Mall Dasma,For Opening,2024-08-24,M7YJ12E,South Luzon,Kathleen Incoy
186,JCE Fone,Infinix Waltermart Bicutan,Waltermart Bicutan,For Opening,2024-08-24,M1KXDBO,South Luzon,Grace Belarmino
185,JCE Fone,Infinix Festival Mall,Festival Mall,For Opening,2024-08-24,MV25D0H,South Luzon,Grace Belarmino
188,JCE Fone,Power Switch Festival Mall,Festival Mall,For Opening,2024-08-24,MIHX81Z,South Luzon,Grace Belarmino
545,JCE Fone,Realme Robinsons Town Los Banos,Robinsons Town Los Banos,For Opening,2024-08-24,M38O0L2,South Luzon,Aubrey Lalong Isip
134,JCE Fone,Infinix Waltermart Trece,Waltermart Trece Martires,For Opening,2024-08-24,MTOSI0A,South Luzon,Kathleen Incoy
544,JCE Fone,Infinix Central Mall Sta Cruz,Central Mall Sta Cruz,For Opening,2024-08-24,MLQ6VA0,South Luzon,Aubrey Lalong Isip
187,JCE Fone,Tecno Waltermart Bicutan,Waltermart Bicutan,For Opening,2024-08-24,MFH3EK5,South Luzon,Grace Belarmino
139,JCE Fone,Infinix LCC Iriga,LCC Mall Iriga,For Opening,2024-08-24,MXUHR78,South Luzon,Penfrancia Ayo


In [46]:
df[df['PurpleKey'] =='M4XIFGS']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
137,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,2024-08-19,M4XIFGS,South Luzon,Penfrancia Ayo


In [47]:
df.groupby(['BrandStores', 'DateOpened'])['PurpleKey'].nunique()

BrandStores                 DateOpened
Affordables                 2024-01-22      1
                            2024-04-12      1
Ansons                      2024-04-15     15
ArkHome                     2024-06-07      4
Automatic Centre            2023-07-15      6
                            2023-07-16      1
                            2023-07-22      4
Chrisrence Home Appliances  2024-07-13      2
Dizon Furniture             2022-07-23      1
ESLA                        2024-02-15     30
                            2024-04-22     21
FC Home                     2023-08-01     48
                            2023-09-01     20
                            2023-09-05      1
                            2023-09-14     25
                            2023-09-15     11
                            2023-11-10      1
                            2024-01-15      1
                            2024-03-26      1
                            2024-04-24      1
                            2024-08-23   

In [48]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
12,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David
13,Rulls,Audionet Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,M8BPO3Y,NCR East,Albee David
15,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
16,Rulls,Rulls Sta Lucia,Sta Lucia Mall,For Opening,2024-04-15,M58WV1L,NCR East,Albee David
17,Rulls,Mobilecraze K1 PG Marikina,Puregold Marikina,For Opening,2024-04-15,M9KLTD1,NCR East,Albee David
18,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
20,Rulls,Techbox Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8FWM1N,NCR East,Albee David
21,Rulls,Oppo Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MTEHX0R,NCR East,Albee David
22,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
24,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David


In [49]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)

In [50]:
df[df['PurpleKey']=='M4XIFGS']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
137,MemoXpress,Jr Mx Waltermart Balayan Batangas,Waltermart Balayan,For Opening,2024-08-19,M4XIFGS,South Luzon,Penfrancia Ayo
